##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 언어 이해를 위한 트랜스포머 

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/text/transformer">
    <img src="https://www.tensorflow.org/images/tf_logo_32px.png" />
    TensorFlow에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/text/transformer.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" />
    구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ko/tutorials/text/transformer.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />
    깃허브(GitHub)소스 보기</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/ko/tutorials/text/transformer.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />노트북에 다운로드</a>
  </td>
</table>

Note: 이 문서는 텐서플로우 커뮤니티에서 번역했습니다. 이 번역에 개선할 부분이 있다면 tensorflow/docs 깃허브 저장소로 풀리퀘스트를 보내주시기 바랍니다. 이 튜토리얼은 포르투칼어를 영어로 번역하기 위해 <a href="https://arxiv.org/abs/1706.03762" class="external">Transformer</a> 모델을 훈련합니다. 이것은 [텍스트 생성](text_generation.ipynb) 및 [주의](nmt_with_attention.ipynb)에 대한 지식을 전제로 한 고급 예시입니다.

트랜스포머 모델의 핵심 아이디어는 *셀프-어텐션*입니다. 즉, 입력 시퀀스의 다른 위치에 참여하여 해당 시퀀스의 표현을 계산할 수 있습니다. 트랜스포머는 셀프-어텐션 계층 스택을 생성하며 아래에서 *스케일 내적 어텐션* 및 *멀티-헤드 어텐션*에 설명되어 있습니다.

트랜스포머 모델은 [RNNs](text_classification_rnn.ipynb) 또는 [CNNs](../images/intro_to_cnns.ipynb) 대신 셀프-어텐션 계층 스택을 사용하여 가변의 크기인 입력을 처리합니다. 이러한 일반적인 아키텍처에는 여러가지 장점이 있습니다:

* 데이터 영역의 시간적/공간적 관계에 대한 가정은 없습니다. 이는 일련의 객체(예: [스타크래프트 유닛](https://deepmind.com/blog/alphastar-mastering-real-time-strategy-game-starcraft-ii/#block-8))를 처리하는데 이상적입니다.
* 계층 출력은 RNN과 같은 시리즈 대신 병렬로 계산할 수 있습니다.
* 먼 항목은 많은 RNN 스텝 또는 합성곱 계층을 거치지 않고 서로의 출력에 영향을 줄 수 있습니다. (예: [Scene 메모리 트랜스포머](https://arxiv.org/pdf/1903.03878.pdf)).
* 장거리 의존성을 배울 수 있습니다. 이것은 많은 시퀀스 작업에서 어려운 문제입니다.

이 아키텍처의 단점은 다음과 같습니다:

* 시계열의 경우, 시간 단계의 출력은 입력 및 현재 숨겨진 상태 대신 *전체 기록*에서 계산됩니다. 이것은 덜 효율적일 _수_ 있습니다.   
* 입력 내용이 텍스트와 같이 시간적/공간적 관계를 *가지는* 경우, 일부 위치 인코딩이 추가되어야 하며 그렇지 않으면 모델이 효과적으로 bag of words을 볼 수 있습니다. 

이 노트에서 모델을 학습한 후, 포르투칼어 문장을 입력하고 영어 번역을 반환할 수 있습니다.

<img src="https://www.tensorflow.org/images/tutorials/transformer/attention_map_portuguese.png" width="800" alt="Attention heatmap">

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow_datasets as tfds
import tensorflow as tf

import time
import numpy as np
import matplotlib.pyplot as plt

## 입력 파이프라인 

[TFDS](https://www.tensorflow.org/datasets)를 사용하여 [TED Talks Open Translation Project](https://www.ted.com/participate/translate)에서 [포르투칼어-영어 번역 데이터셋](https://github.com/neulab/word-embeddings-for-nmt)을 로드합니다.

이 데이터셋은 약 50000개의 training examples, 1100개의 validation examples 그리고 2000개의 test examples가 포함됩니다.

In [0]:
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)
train_examples, val_examples = examples['train'], examples['validation']

트레이닝 데이터셋에서 사용자 정의 하위 단어 토크나이저를 생성합니다. 

In [0]:
tokenizer_en = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (en.numpy() for pt, en in train_examples), target_vocab_size=2**13)

tokenizer_pt = tfds.features.text.SubwordTextEncoder.build_from_corpus(
    (pt.numpy() for pt, en in train_examples), target_vocab_size=2**13)

In [0]:
sample_string = 'Transformer is awesome.'

tokenized_string = tokenizer_en.encode(sample_string)
print ('토큰화 된 문자열 : {}'.format(tokenized_string))

original_string = tokenizer_en.decode(tokenized_string)
print ('원본 문자열 : {}'.format(original_string))

assert original_string == sample_string

토크나이저는 단어가 사전에 없는 경우 문자열을 하위 단어로 분할하여 인코딩합니다.

In [0]:
for ts in tokenized_string:
  print ('{} ----> {}'.format(ts, tokenizer_en.decode([ts])))

In [0]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

입력 및 대상에 시작 및 종료 토큰을 추가합니다. 

In [0]:
def encode(lang1, lang2):
  lang1 = [tokenizer_pt.vocab_size] + tokenizer_pt.encode(
      lang1.numpy()) + [tokenizer_pt.vocab_size+1]

  lang2 = [tokenizer_en.vocab_size] + tokenizer_en.encode(
      lang2.numpy()) + [tokenizer_en.vocab_size+1]
  
  return lang1, lang2

Note: 이 예제를 작고 비교적 빠르게 유지하기 위해서 토큰 길이가 40개가 넘는 예제를 삭제합니다.

In [0]:
MAX_LENGTH = 40

In [0]:
def filter_max_length(x, y, max_length=MAX_LENGTH):
  return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)

`.map()` 내부의 작업은 그래프 모드에서 실행되며 numpy 속성이 없는 그래프 텐서를 받습니다. `토크나이저`는 문자열 또는 유니코드 기호가 이것을 정수로 인코딩 할 것으로 예상합니다. 따라서, 문자열 값을 포함하는 numpy 속성을 가진 텐서를 수신하는 `tf.py_function` 내부에서 인코딩을 실행해야 합니다.

In [0]:
def tf_encode(pt, en):
  return tf.py_function(encode, [pt, en], [tf.int64, tf.int64])

In [0]:
train_dataset = train_examples.map(tf_encode)
train_dataset = train_dataset.filter(filter_max_length)
# 데이터를 읽는 동안 속도를 높이려면 데이터 세트를 메모리에 캐시해야 합니다.
train_dataset = train_dataset.cache()
train_dataset = train_dataset.shuffle(BUFFER_SIZE).padded_batch(
    BATCH_SIZE, padded_shapes=([-1], [-1]))
train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)


val_dataset = val_examples.map(tf_encode)
val_dataset = val_dataset.filter(filter_max_length).padded_batch(
    BATCH_SIZE, padded_shapes=([-1], [-1]))

In [0]:
pt_batch, en_batch = next(iter(val_dataset))
pt_batch, en_batch

## 위치 인코딩

이 모델에는 반복이나 컨볼루션이 포함되어 있지 않으므로 위치 인코딩이 추가되어 문장에서 단어의 상대 위치에 대한 정보를 모델에 제공합니다. 

위치 인코딩 벡터가 임베딩 벡터에 추가됩니다. 임베딩은 비슷한 의미의 토큰이 서로 더 가까운 d-차원 공간에서의 토큰을 나타냅니다. 그러나 임베딩은 문장에서 단어의 상대적 위치를 인코딩하지 않습니다. 따라서 위치 인코딩을 추가한 후 d-차원 공간에서 *문장의 의미와 문장에서의 위치의 유사성*에 따라 단어가 서로 더 가까워집니다.

 [위치 인코딩](https://github.com/tensorflow/examples/blob/master/community/en/position_encoding.ipynb)에 대한 자세한 내용은 노트북을 참고하길 바랍니다. 위치 인코딩을 계산하는 공식은 다음과 같습니다:

$$\Large{PE_{(pos, 2i)} = sin(pos / 10000^{2i / d_{model}})} $$
$$\Large{PE_{(pos, 2i+1)} = cos(pos / 10000^{2i / d_{model}})} $$

In [0]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

In [0]:
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)
  
  # 배열의 짝수 인덱스에 sin을 적용합니다; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
  
  # 배열의 홀수 인덱스에 cos를 적용합니다; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    
  pos_encoding = angle_rads[np.newaxis, ...]
    
  return tf.cast(pos_encoding, dtype=tf.float32)

In [0]:
pos_encoding = positional_encoding(50, 512)
print (pos_encoding.shape)

plt.pcolormesh(pos_encoding[0], cmap='RdBu')
plt.xlabel('Depth')
plt.xlim((0, 512))
plt.ylabel('Position')
plt.colorbar()
plt.show()

## 마스킹

순서대로 모든 패드 토큰을 마스크합니다. 모델이 패딩을 입력으로 처리하지 않도록 합니다. 마스크는 패드 값 `0`이 존재한느 위치를 나타냅니다. 해당 위치에서 `1`을, 그렇지 않으면 `0`을 출력합니다.

In [0]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
  
  # 치수를 추가하여 어텐션 로그에
  # 패딩을 추가합니다.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [0]:
x = tf.constant([[7, 6, 0, 0, 1], [1, 2, 3, 0, 0], [0, 0, 0, 4, 5]])
create_padding_mask(x)

미리보기 마스크는 미래의 토큰을 순서대로 마스킹하는데 사용합니다. 다시 말해서, 마스크는 어떠한 항목을 사용하지 않아야 하는지 나타냅니다.

이는 세 번째 단어를 예측하기 위해, 첫 번째와 두 번째 단어만 사용된다는 것을 의미합니다. 네 번째 단어를 예측하는 것과 마찬가지로, 첫 번째, 두 번째 및 세 번째 단어만 사용됩니다.

In [0]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

In [0]:
x = tf.random.uniform((1, 3))
temp = create_look_ahead_mask(x.shape[1])
temp

## 스케일 내적 어텐션

<img src="https://www.tensorflow.org/images/tutorials/transformer/scaled_attention.png" width="500" alt="scaled_dot_product_attention">

트랜스포머에서 사용되는 어텐션 기능은 Q(쿼리), K(키), V(값)의 세 가지 입력이 있습니다. 어텐션 가중치를 계산하는데 사용되는 방정식은 다음과 같습니다:

$$\Large{Attention(Q, K, V) = softmax_k(\frac{QK^T}{\sqrt{d_k}}) V} $$

내적 어텐션은 깊이의 제곱근에 의해 조정됩니다. 이는 깊이 값이 클 수록 내적의 크기가 크게 거대해져 기울기가 작은 소프트맥스 함수를 사용하여 소프트 맥스가 매우 강력해지기 때문입니다. 

예를 들어, `Q`와 `K`의 평균이 0이고 분산이 1이라고 가정합니다. 행렬곱의 평균은 0이고 분산은 `dk`입니다. 따라서, `Q`와 `K`의 행렬곱의 평균은 0이고 분산은 1이어야 하므로 *`dk`의 제곱근*은 스케일링에 사용됩니다.(다른 숫자는 아닙니다).

마스크에는 -1e9가 곱해집니다(이 값은 음의 무한대에 가깝습니다). 마스크가 Q와 K의 스케일된 행렬곱과 합쳐져서 소프트맥스 직전에 적용되기 때문에 수행됩니다. 목표는 이러한 셀을 제로화하는 것이며, 소프트맥스에 대한 큰 음의 입력은 출력에서 거의 제로에 가깝습니다.

In [0]:
def scaled_dot_product_attention(q, k, v, mask):
  """어텐션 가중치를 계산합니다.
  q, k, v는 선행 치수와 일치해야 합니다.
  k, v는 일치하는 두 번째 차원을 가져야합니다, 예: seq_len_k = seq_len_v.
  마스크는 종류에 따라 모양이 다릅니다(패딩 또는 미리보기) 
  하지만 추가하려면 브로드 캐스트를 해야 합니다.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable 
          to (..., seq_len_q, seq_len_k). Defaults to None.
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # 스케일 텐서에 마스크를 추가합니다.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)  

  # 소프트맥스는 마지막 축(seq_len_k)에서 정규화되므로 점수가
  # 1이 됩니다.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

소프트맥스 정규화가 K에서 수행되 때, 그 값은 Q에 주어진 중요도를 결정하게 됩니다.

출력은 어텐션 가중치와 V(값) 벡터의 곱을 나타냅니다. 이렇게 하면 집중하려는 단어가 그대로 유지되고 관련이 없는 단어는 플러시됩니다.

In [0]:
def print_out(q, k, v):
  temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
  print ('어텐션 가중치:')
  print (temp_attn)
  print ('출력값:')
  print (temp_out)

In [0]:
np.set_printoptions(suppress=True)

temp_k = tf.constant([[10,0,0],
                      [0,10,0],
                      [0,0,10],
                      [0,0,10]], dtype=tf.float32)  # (4, 3)

temp_v = tf.constant([[   1,0],
                      [  10,0],
                      [ 100,5],
                      [1000,6]], dtype=tf.float32)  # (4, 2)

# 이 `쿼리`는 두 번째 `키`와 정렬되므로,
# 두 번째`값`이 반환됩니다.
temp_q = tf.constant([[0, 10, 0]], dtype=tf.float32)  # (1, 3)
print_out(temp_q, temp_k, temp_v)

In [0]:
# 이쿼리는 반복되는 키(3 및 4)와 정렬되므로, 
# 모든 관련 값이 평균화됩니다.
temp_q = tf.constant([[0, 0, 10]], dtype=tf.float32)  # (1, 3)
print_out(temp_q, temp_k, temp_v)

In [0]:
# 이 쿼리는 첫 번째와 두 번째 키와 동일하게 정렬되므로, 
# 값이 평균화됩니다.
temp_q = tf.constant([[10, 10, 0]], dtype=tf.float32)  # (1, 3)
print_out(temp_q, temp_k, temp_v)

모든 쿼리를 함께 전달합니다.

In [0]:
temp_q = tf.constant([[0, 0, 10], [0, 10, 0], [10, 10, 0]], dtype=tf.float32)  # (3, 3)
print_out(temp_q, temp_k, temp_v)

## 멀티-헤드 어텐션

<img src="https://www.tensorflow.org/images/tutorials/transformer/multi_head_attention.png" width="500" alt="multi-head attention">


멀티-헤드 어텐션은 네 가지 부분으로 구성됩니다:
*    선형 계층과 헤드로 분할.
*    스케일 내적 어텐션.
*    헤드의 연결.
*    최종 선형 계층.

각 멀티-헤드 어텐션에는 3개의 입력이 있습니다; Q(쿼리), K(키), V(값). 이들은 선형(고밀도) 계층을 통해 여러 헤드로 분할됩니다. 

위에서 정의한`스케일 내적 어텐션`은 각 헤드에 적용됩니다(효율성을 위한 브로드캐스트). 어텐션 단계에서 적절한 마스크를 사용해야 합니다. 그런 다음 각 헤드에 대한 어텐션 출력이 연결되고(`tf.transpose`, 및 `tf.reshape`를 사용하여) 최종 `밀도` 계층을 통과합니다.

단일 어텐션 헤드 대신, Q, K, 및 V는 여러 헤드로 분할되어 모델이 서로 다른 표현 공간의 다른 위치에 있는 정보에 공동으로 참여할 수 있게 합니다. 분할 후 각 헤드의 치수가 감소하므로 전체 계산 비용은 전체 치수로 단일 헤드 어텐션과 동일합니다.

In [0]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """마지막 차원을 (num_heads, depth)로 분할합니다.
    모양이 (batch_size, num_heads, seq_len, depth)가 되도록 결과를 바꿉니다.
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

시험해 볼 `MultiHeadAttention` 계층을 만듭니다. 시퀀스의 각 위치, `y`,에서 `MultiHeadAttention`은 시퀀스의 다른 모든 위치에서 8개의 어텐션 헤드를 모두 실행하여 각 위치에서 동일한 길이의 새로운 벡터를 반환합니다.

In [0]:
temp_mha = MultiHeadAttention(d_model=512, num_heads=8)
y = tf.random.uniform((1, 60, 512))  # (batch_size, encoder_sequence, d_model)
out, attn = temp_mha(y, k=y, q=y, mask=None)
out.shape, attn.shape

## 포인트 단위 피드 포워드 네트워크

포인트 단위 피드 포워드 네트워크는 ReLU 활성화함수를 통해 두 개의 완전 연결 계층으로 구성됩니다.

In [0]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [0]:
sample_ffn = point_wise_feed_forward_network(512, 2048)
sample_ffn(tf.random.uniform((64, 50, 512))).shape

## 인코더와 디코더

<img src="https://www.tensorflow.org/images/tutorials/transformer/transformer.png" width="600" alt="transformer">

트랜스포머 모델은 표준 시퀀스와 동일한 일반 패턴을 따라 [어텐션 모델로 시퀀싱](nmt_with_attention.ipynb)됩니다. 

* 입력 문장은 시퀀스의 각 단어/토큰에 대한 출력을 생성하는`N`개의 인코더 계층을 통과합니다.
* 디코더는 다음 단어를 예측하기 위해 인코더의 출력과 자체 입력(셀프-어텐션)에 참여합니다. 

### 인코더 계층

각각의 인코더 계층은 서브계층들로 구성됩니다:

1.   멀티-헤드 어텐션 (패딩 마스크 포함) 
2.    점 단위 피드 포워드 네트워크. 

이 서브 계층들의 각각은 그 주위에 잔류 연결을 갖고 연결하여 계층 정규화를 가집니다. 잔여 연결은 딥 네트워크에서 사라지는 그레디언트 문제를 방지하는데 도움이 됩니다.

각 하위 계층의 출력은 `LayerNorm(x + Sublayer(x))`입니다. 정규화는 `d_model`(마지막) 축에서 수행됩니다. 트랜스포머에는 N개의 인코더 계층이 있습니다.

In [0]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out2

In [0]:
sample_encoder_layer = EncoderLayer(512, 8, 2048)

sample_encoder_layer_output = sample_encoder_layer(
    tf.random.uniform((64, 43, 512)), False, None)

sample_encoder_layer_output.shape  # (batch_size, input_seq_len, d_model)

### 디코더 계층

각각의 디코더 계층은 서브계층들로 구성됩니다:

1.   마스킹된 멀티-헤드 어텐션 (미리보기 마스크 및 패딩 마스크 포함)
2.   멀티-헤드 어텐션(패딩 마스크 포함). V(값) 및 K(키)는 *인코더 출력*을 입력드로 받습니다. Q(쿼리)는 *마스킹된 멀티-헤드 어텐션의 서브 계층.*으로부터 출력을 수신합니다.
3.   점 단위 피드 포워드 네트워크

이러한 서브 계층 각각은 그 주위에 잔류 연결을 갖고 연결하여 계층 정규화를 가집니다. 각 하위 계층의 출력은 `LayerNorm(x + Sublayer(x))`입니다. 정규화는 `d_model`(마지막) 축에서 수행됩니다.

트랜스포머에는 N개의 디코더 계층이 있습니다.

Q가 디코더의 첫 번째 어텐션 블록에서 출력을 수신하고 K가 인코더 출력을 수신함에 따라 어텐션 가중치는 인코더의 출력을 기반으로 디코더의 입력에 주어진 중요성을 나타냅니다. 다시말해, 디코더는 인코더 출력을 보고 자체 출력에 자체-참여하여 다음 단어를 예측합니다. 스케일 내적 어텐션 섹션에서 위의 데모를 참조하길 바랍니다.

In [0]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)
 
    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)
    
    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)
    
    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)
    
    return out3, attn_weights_block1, attn_weights_block2

In [0]:
sample_decoder_layer = DecoderLayer(512, 8, 2048)

sample_decoder_layer_output, _, _ = sample_decoder_layer(
    tf.random.uniform((64, 50, 512)), sample_encoder_layer_output, 
    False, None, None)

sample_decoder_layer_output.shape  # (batch_size, target_seq_len, d_model)

### 인코더

`인코더`를 구성하는 것들:
1.   입력 임베딩
2.   위치 인코딩
3.   N 개의 인코더 계층

입력은 위치 인코딩과 합쳐진 임베딩을 통해 이루어집니다. 이러한 합산의 출력은 인코더 계층에 대한 입력입니다. 인코더의 출력은 디코더에 대한 입력입니다.

In [0]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)
    
    
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
  
    self.dropout = tf.keras.layers.Dropout(rate)
        
  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]
    
    # 임베딩과 위치 인코딩을 추가합니다.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    
    return x  # (batch_size, input_seq_len, d_model)

In [0]:
sample_encoder = Encoder(num_layers=2, d_model=512, num_heads=8, 
                         dff=2048, input_vocab_size=8500,
                         maximum_position_encoding=10000)
temp_input = tf.random.uniform((64, 62), dtype=tf.int64, minval=0, maxval=200)

sample_encoder_output = sample_encoder(temp_input, training=False, mask=None)

print (sample_encoder_output.shape)  # (batch_size, input_seq_len, d_model)

### 디코더

 `디코더`를 구성하는 것들:
1.   출력 임베딩
2.   위치 인코딩
3.   N 개의 디코더 계층

타겟은 위치 인코딩과 합산된 임베딩을 통해 이루어집니다. 이러한 합산의 출력은 디코더 계층에 대한 입력입니다. 디코더의 출력은 최종 선형 계층에 대한 입력입니다.

In [0]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
    
    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)
    
  def call(self, x, enc_output, training, 
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}
    
    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]
    
    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)
      
      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [0]:
sample_decoder = Decoder(num_layers=2, d_model=512, num_heads=8, 
                         dff=2048, target_vocab_size=8000,
                         maximum_position_encoding=5000)
temp_input = tf.random.uniform((64, 26), dtype=tf.int64, minval=0, maxval=200)

output, attn = sample_decoder(temp_input, 
                              enc_output=sample_encoder_output, 
                              training=False,
                              look_ahead_mask=None, 
                              padding_mask=None)

output.shape, attn['decoder_layer2_block2'].shape

## 트랜스포머 만들기

트랜스포머는 인코더, 디코더 및 최종 선형 계층으로 구성됩니다. 디코더의 출력은 선형 계층에 대한 입력이며 출력이 반환됩니다.

In [0]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, 
               target_vocab_size, pe_input, pe_target, rate=0.1):
    super(Transformer, self).__init__()

    self.encoder = Encoder(num_layers, d_model, num_heads, dff, 
                           input_vocab_size, pe_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff, 
                           target_vocab_size, pe_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
  def call(self, inp, tar, training, enc_padding_mask, 
           look_ahead_mask, dec_padding_mask):

    enc_output = self.encoder(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)
    
    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)
    
    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)
    
    return final_output, attention_weights

In [0]:
sample_transformer = Transformer(
    num_layers=2, d_model=512, num_heads=8, dff=2048, 
    input_vocab_size=8500, target_vocab_size=8000, 
    pe_input=10000, pe_target=6000)

temp_input = tf.random.uniform((64, 38), dtype=tf.int64, minval=0, maxval=200)
temp_target = tf.random.uniform((64, 36), dtype=tf.int64, minval=0, maxval=200)

fn_out, _ = sample_transformer(temp_input, temp_target, training=False, 
                               enc_padding_mask=None, 
                               look_ahead_mask=None,
                               dec_padding_mask=None)

fn_out.shape  # (batch_size, tar_seq_len, target_vocab_size)

## 하이퍼파라미터 설정

이 예제를 작고 비교적 빠르게 유지하기 위해서 *num_layers, d_model, 및 dff*의 값을 줄였습니다.

트랜스포머의 기본 모델에서 사용된 값은 다음과 같습니다; *num_layers=6*, *d_model = 512*, *dff = 2048*. 다른 모든 버전의 트랜스포머에 대해서는 [논문](https://arxiv.org/abs/1706.03762)을 참조하길 바랍니다.

Note: 아래 값을 변경하면, 많은 작업에서 최첨단 모델을 얻을 수 있습니다.

In [0]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8

input_vocab_size = tokenizer_pt.vocab_size + 2
target_vocab_size = tokenizer_en.vocab_size + 2
dropout_rate = 0.1

## 최적화

[논문](https://arxiv.org/abs/1706.03762)의 공식에 따라 맞춤형 학습 속도 스케줄러와 함께 Adam optimizer를 사용하길 바랍니다.

$$\Large{lrate = d_{model}^{-0.5} * min(step{\_}num^{-0.5}, step{\_}num * warmup{\_}steps^{-1.5})}$$


In [0]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps
    
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)
    
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [0]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [0]:
temp_learning_rate_schedule = CustomSchedule(d_model)

plt.plot(temp_learning_rate_schedule(tf.range(40000, dtype=tf.float32)))
plt.ylabel("Learning Rate")
plt.xlabel("Train Step")

## 손실과 매트릭스

타겟 시퀀스가 채워지기 때문에, 손실을 계산할 때 패딩 마스크를 적용하는 것이 중요합니다.

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [0]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask
  
  return tf.reduce_mean(loss_)

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(
    name='train_accuracy')

## 훈련과 체크포인팅

In [0]:
transformer = Transformer(num_layers, d_model, num_heads, dff,
                          input_vocab_size, target_vocab_size, 
                          pe_input=input_vocab_size, 
                          pe_target=target_vocab_size,
                          rate=dropout_rate)

In [0]:
def create_masks(inp, tar):
  # 인코더 패딩 마스크
  enc_padding_mask = create_padding_mask(inp)
  
  # 디코더의 두 번째 어텐션 블록에 사용됩니다.
  # 이 패딩 마스크는 인코더 출력을 마스킹하는데 사용됩니다.
  dec_padding_mask = create_padding_mask(inp)
  
  # 디코더의 첫 번째 어텐션 블록에 사용됩니다.
  # 디코더가 수신한 입력에서 미래 토큰을 채우고 마스킹하는데 
  # 사용됩니다.
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
  return enc_padding_mask, combined_mask, dec_padding_mask

체크포인트 경로와 체크포인트 관리자를 만듭니다. 이것은 `n` 에포크마다 체크포인트를 저장하는데 사용됩니다.

In [0]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# 체크포인트가 존재한다면, 최신 체크포인트로 복원합니다.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('최신 체크포인트가 복원되었습니다!!')

대상은 tar_inp와 tar_real로 나뉩니다. tar_inp는 디코더에 입력으로 전달 됩니다. `tar_real`은 동일한 입력이 1만큼 이동된 것입니다:  `tar_input`의 각 위치에서, `tar_real`은 예측해야 할 다음 토큰을 포함합니다.

예를 들어, `문장` = "SOS 한 사자가 정글에서 자고 있습니다 EOS"

`tar_inp` =  "SOS 한 사자가 정글에서 자고 있습니다"

`tar_real` = "한 사자가 정글에서 자고 있습니다 EOS"

트랜스포머는 자동-회기 모델입니다: 한 번에 한 부분씩 예측을 하고, 그 결과를 사용하여 다음에 수행 할 작업을 결정합니다. 

훈련하는 동안 이 예제는 teacher-forcing을 사용합니다 ([텍스트 생성 튜토리얼](./text_generation.ipynb)에서와 같이). Teacher forcing은 현재 시간 단계에서 모델이 예측하는 것과 상관없이 실제 출력을 다음 시간 단계로 전달합니다.

트랜스포머는 각 단어를 예측할 때, *셀프-어텐션*을 통해 입력 시퀀스에서 이전 단어를 보고 다음 단어를 더 잘 예측할 수 있습니다.

모델이 예상 출력에서 정점에 도달하는 것을 방지하기 위해 모델은 미리보기 마스크를 사용합니다.

In [0]:
EPOCHS = 20

In [0]:
# @tf.function은 train_step을 더 빠른 실행을 위해 TF 그래프로 추적
# 컴파일합니다. 이 함수는 인수 텐서의 정확한 모양에 특화되어
# 있습니다. 가변의 시퀀스 길이 또는 가변의 배치 크기
# (마지막 배치가 더 작음)로 인한 재추적을 피하려면 input_signature를 사용하여 보다
# 일반적인 모양을 지정해야 합니다.

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]

@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]
  
  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)
  
  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp, 
                                 True, 
                                 enc_padding_mask, 
                                 combined_mask, 
                                 dec_padding_mask)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
  
  train_loss(loss)
  train_accuracy(tar_real, predictions)

입력 언어로는 포르투칼어가 사용되고 영어는 타켓 언어입니다.

In [0]:
for epoch in range(EPOCHS):
  start = time.time()
  
  train_loss.reset_states()
  train_accuracy.reset_states()
  
  # inp -> portuguese, tar -> english
  for (batch, (inp, tar)) in enumerate(train_dataset):
    train_step(inp, tar)
    
    if batch % 50 == 0:
      print ('에포크 {} 배치 {} 손실 {:.4f} 정확도 {:.4f}'.format(
          epoch + 1, batch, train_loss.result(), train_accuracy.result()))
      
  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print ('체크포인트 저장 for epoch {} at {}'.format(epoch+1,
                                                         ckpt_save_path))
    
  print ('에포크 {} 손실 {:.4f} 정확도 {:.4f}'.format(epoch + 1, 
                                                train_loss.result(), 
                                                train_accuracy.result()))

  print ('1 에포크에 걸린 시간: {} secs\n'.format(time.time() - start))

## 평가하기

다음 단계는 평가에 사용됩니다:

* 포르투갈어 토크나이저(`tokenizer_pt`)를 사용하여 입력 문장을 인코딩합니다. 또한, 시작 및 종료 토큰을 추가하여 입력이 모델이 학습 한 것과 동일하도록 합니다. 이것은 인코더 입력입니다.
* 디코더 입력은 `start token == tokenizer_en.vocab_size`입니다.
* 패딩 마스크와 미리보기 마스크를 계산합니다.
* 그런 다음 `디코더`는 `인코더 출력`과 자체 출력(셀프-어텐션)을 보고 예측을 출력합니다.
* 마지막 단어를 선택하고 해당 당어의 argmax를 계산합니다.
* 예측된 단어를 디코더 입력에 연결하여 디코더로 전달합니다.
* 이 접근법에서, 디코더는 예측된 이전 단어에 기초하여 다음 단어를 예측합니다.

Note: 여기에 사용된 모델은 예제를 상대적으로 빠르게 유지할 수 있는 용량이 적으므로 예측이 덜 적합할 수 있습니다. 논문의 결과를 재현하려면 위의 하이퍼 파라미터를 변경하여 전체 데이터 세트 및 기본 트랜스포머 모델 또는 트랜스포머 XL을 사용합니다.

In [0]:
def evaluate(inp_sentence):
  start_token = [tokenizer_pt.vocab_size]
  end_token = [tokenizer_pt.vocab_size + 1]
  
  # inp 문장은 포르투갈어이므로 시작 및 끝 토큰을 추가합니다
  inp_sentence = start_token + tokenizer_pt.encode(inp_sentence) + end_token
  encoder_input = tf.expand_dims(inp_sentence, 0)
  
  # 타겟이 영어이므로 트랜스포머의 첫 번째 단어는
  # 영어 시작 토큰이어야 합니다.
  decoder_input = [tokenizer_en.vocab_size]
  output = tf.expand_dims(decoder_input, 0)
    
  for i in range(MAX_LENGTH):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)
  
    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(encoder_input, 
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)
    
    # seq_len 차원에서 마지막 단어를 선택합니다
    predictions = predictions[: ,-1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)
    
    # predict_id가 종료 토큰과 같은 경우 결과를 반환
    if predicted_id == tokenizer_en.vocab_size+1:
      return tf.squeeze(output, axis=0), attention_weights
    
    # prediected_id를 입력으로서 디코더에 제공되는
    # 출력에 연결합니다.
    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0), attention_weights

In [0]:
def plot_attention_weights(attention, sentence, result, layer):
  fig = plt.figure(figsize=(16, 8))
  
  sentence = tokenizer_pt.encode(sentence)
  
  attention = tf.squeeze(attention[layer], axis=0)
  
  for head in range(attention.shape[0]):
    ax = fig.add_subplot(2, 4, head+1)
    
    # 어텐션 가중치 plot
    ax.matshow(attention[head][:-1, :], cmap='viridis')

    fontdict = {'fontsize': 10}
    
    ax.set_xticks(range(len(sentence)+2))
    ax.set_yticks(range(len(result)))
    
    ax.set_ylim(len(result)-1.5, -0.5)
        
    ax.set_xticklabels(
        ['<start>']+[tokenizer_pt.decode([i]) for i in sentence]+['<end>'], 
        fontdict=fontdict, rotation=90)
    
    ax.set_yticklabels([tokenizer_en.decode([i]) for i in result 
                        if i < tokenizer_en.vocab_size], 
                       fontdict=fontdict)
    
    ax.set_xlabel('Head {}'.format(head+1))
  
  plt.tight_layout()
  plt.show()

In [0]:
def translate(sentence, plot=''):
  result, attention_weights = evaluate(sentence)
  
  predicted_sentence = tokenizer_en.decode([i for i in result 
                                            if i < tokenizer_en.vocab_size])  

  print('입력: {}'.format(sentence))
  print('예측된 번역: {}'.format(predicted_sentence))
  
  if plot:
    plot_attention_weights(attention_weights, sentence, result, plot)

In [0]:
translate("este é um problema que temos que resolver.")
print ("실제 번역: this is a problem we have to solve .")

In [0]:
translate("os meus vizinhos ouviram sobre esta ideia.")
print ("실제 번역: and my neighboring homes heard about this idea .")

In [0]:
translate("vou então muito rapidamente partilhar convosco algumas histórias de algumas coisas mágicas que aconteceram.")
print ("실제 번역: so i 'll just share with you some stories very quickly of some magical things that have happened .")

디코더의 다른 계층 및 어텐션 블록을 `plot` 매개변수로 전달할 수 있습니다.

In [0]:
translate("este é o primeiro livro que eu fiz.", plot='decoder_layer4_block2')
print ("실제 번역: this is the first book i've ever done.")

## 

이번 튜토리얼에서, 위치 인코딩, 멀티-헤드 어텐션, 마스킹의 중요성 및 트랜스포머 작성 방법에 대해 배웠습니다.

트랜스포머를 훈련시키기 위해 다른 데이터 세트를 사용해보길 권장합니다. 위의 하이퍼 파라미터를 변경하여 기본 트랜스포머 혹은 트랜스포머 XL을 생성할 수도 있습니다. 이곳에 정의된 계층을 사용하여 [BERT](https://arxiv.org/abs/1810.04805)를 만들고 최신 모델을 훈련시킬 수 있습니다. 또한 빔 검색을 구현하여 더 나은 예측을 얻을 수 있습니다.